In [160]:
from erpe.analysis import *
from erpe.models import *
import itertools as itt
import numpy as np
import math
from pygsti.circuits import Circuit

# autorload 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [161]:
ZZ_centralizer = [ 
    'XX', 'YY', 'ZZ', 'XY', 'YX', 'IZ', 'ZI', 'II'
]

# make all orderings of the centralizer
ZZ_centralizer_permutations = list(itt.permutations(ZZ_centralizer))
print(len(ZZ_centralizer_permutations) == math.factorial(8))

True


In [162]:
def onequbit_pauli_to_pygsti_gate(p, qid):
    if p == 'X':
        return Circuit([('Gxpi2', qid)]*2, line_labels=qids)
    elif p == 'Y':
        return Circuit([('Gypi2', qid)]*2, line_labels=qids)
    elif p == 'Z':
        return Circuit([('Gzpi2', qid)]*2, line_labels=qids)
    elif p == 'I':
        return Circuit([], line_labels=qids)

def twoqubit_pauli_to_pygsti_gates(term, qids):
    gates = Circuit(line_labels=qids)
    gates += onequbit_pauli_to_pygsti_gate(term[0], qid=qids[0])
    gates += onequbit_pauli_to_pygsti_gate(term[1], qid=qids[1])
    return gates


def decoupling_sequence_to_pygsti_circ(sequence, prep, idle_rep_interval, qids):
    circ = Circuit(line_labels=qids)
    if prep == 'I':
        circ += Circuit([('Gypi2', qids[0])])
    if prep == 'Q':
        circ += Circuit([('Gxpi2', qids[0])])
    for term in sequence:
        gates = twoqubit_pauli_to_pygsti_gates(term, qids)
        circ += gates
        circ += Circuit([('Gidle', *qids)])*idle_rep_interval
        circ += gates
    circ += Circuit([('Gypi2', qids[0])])*3
    return circ
        

In [198]:
xvec = 1e-3*np.random.randn(15) + 0.01*np.eye(15)[-1]
qids = ['Q0', 'Q1']
model = create_idle_model(xvec, qids)
circ = decoupling_sequence_to_pygsti_circ(ZZ_centralizer_permutations[4], 'Q', 1, qids)
print(model.probabilities(circ))

OutcomeLabelDict({('00',): np.float64(0.5411198297430844), ('01',): np.float64(1.3048458147313369e-09), ('10',): np.float64(0.45888016821976013), ('11',): np.float64(7.323091022470862e-10)})


# twirled RPE

1) build up a list of circuits
- the circuit list will be a 2d list of circuits at each detph
2) sample from the circuits
3) perform data analysis



In [199]:
# circuit list buildup 

num_randomizations_per_depth = 10
num_shots_per_circuit = 64
depths = [2**i for i in range(8)]

Icircs = []
Qcircs = []
for d in depths: 
    Icircs_at_depth = []
    Qcircs_at_depth = []
    for r in range(num_randomizations_per_depth):
        # pick a random permutation of the commutant 
        perm_idx = np.random.choice(range(len(ZZ_centralizer_permutations)))
        perm = ZZ_centralizer_permutations[perm_idx]
        Icirc = decoupling_sequence_to_pygsti_circ(perm, 'I', d, qids)
        Icircs_at_depth.append(Icirc)
        Qcirc = decoupling_sequence_to_pygsti_circ(perm, 'Q', d, qids)
        Qcircs_at_depth.append(Qcirc)
    Icircs.append(Icircs_at_depth)
    Qcircs.append(Qcircs_at_depth)


In [200]:
all_circs_needing_data = []
for d in range(len(depths)):
    all_circs_needing_data += Icircs[d]
    all_circs_needing_data += Qcircs[d]
len(all_circs_needing_data)

160

In [201]:
ds = pygsti.data.simulate_data(model, all_circs_needing_data, num_shots_per_circuit)

In [202]:
def extract_counts_from_dataset(dataset, Icircs, Ioutcomes, Qcircs, Qoutcomes, depths):
    Icounts_at_depth = []
    Qcounts_at_depth = []
    for d in range(len(depths)):
        Icircs_at_depth = Icircs[d]
        Qcircs_at_depth = Qcircs[d]
        Icounts = np.array([0., 0.])
        Qcounts = np.array([0., 0.])
        for Icirc in Icircs_at_depth:
            plus_outcome = dataset[Icirc][Ioutcomes[0]]
            minus_outcome = dataset[Icirc][Ioutcomes[1]]
            Icounts += np.array([plus_outcome, minus_outcome])
        for Qcirc in Qcircs_at_depth:
            plus_outcome = dataset[Qcirc][Qoutcomes[0]]
            minus_outcome = dataset[Qcirc][Qoutcomes[1]]
            Qcounts += np.array([plus_outcome, minus_outcome])
        Icounts_at_depth.append(Icounts)
        Qcounts_at_depth.append(Qcounts)
    return np.array(Icounts_at_depth), np.array(Qcounts_at_depth)

In [203]:
Icounts, Qcounts = extract_counts_from_dataset(ds, Icircs, ['00', '10'], Qcircs, ['00', '10'], depths)
cos_plus_counts = Icounts[:, 0]
cos_minus_counts = Icounts[:, 1]
sin_plus_counts = Qcounts[:, 0]
sin_minus_counts = Qcounts[:, 1]
print(estimate_phase(depths, cos_plus_counts, cos_minus_counts, sin_plus_counts, sin_minus_counts))
print(xvec[-1]*8)

(array([0.06553103, 0.11201182, 0.082637  , 0.08464095, 0.08243324,
       0.08132479, 0.08202666, 0.08306738]), 7)
0.08233151392154694


# compare with naive solution

In [204]:
def make_naive_circ(prep, idle_rep, qids):
    circ = Circuit(line_labels=qids)
    if prep == 'I':
        circ += Circuit([('Gypi2', qids[0])])
    if prep == 'Q':
        circ += Circuit([('Gxpi2', qids[0])])
    circ += Circuit([('Gidle', *qids)])*idle_rep
    circ += Circuit([('Gypi2', qids[0])])*3
    return circ

In [205]:
naive_I_circs = [make_naive_circ('I', d, qids) for d in depths]
naive_Q_circs = [make_naive_circ('Q', d, qids) for d in depths]
all_naive_circs_needing_data = naive_I_circs + naive_Q_circs
naive_ds = pygsti.data.simulate_data(model, all_naive_circs_needing_data, num_shots_per_circuit*num_randomizations_per_depth)
naive_I_counts = np.array([[naive_ds[circ]['00'], naive_ds[circ]['10']] for circ in naive_I_circs])
naive_Q_counts = np.array([[naive_ds[circ]['00'], naive_ds[circ]['10']] for circ in naive_Q_circs])

naive_cos_plus_counts = naive_I_counts[:, 0]
naive_cos_minus_counts = naive_I_counts[:, 1]
naive_sin_plus_counts = naive_Q_counts[:, 0]
naive_sin_minus_counts = naive_Q_counts[:, 1]
print(estimate_phase(depths, naive_cos_plus_counts, naive_cos_minus_counts, naive_sin_plus_counts, naive_sin_minus_counts))
print(xvec[-1]*8)

(array([6.28006032, 0.03742992, 0.01482634, 0.0179572 , 0.01568767,
       0.01202516, 0.01211351, 0.0120247 ]), 7)
0.08233151392154694
